# Discovering the Parent Hamiltonian of a Perfect MPS via Optimization

## Overview
This notebook simulates a sophisticated scenario where we have a quantum state that is guaranteed to be a perfect Matrix Product State (MPS), and our goal is to discover its parent Hamiltonian using a variational, optimization-based approach.

The process is as follows:
1.  **Generate a Perfect MPS State:** We create `|psi_initial>` by directly contracting a chain of random tensors. This ensures our "unknown" state has an exact MPS representation with a known `BOND_DIMENSION`.
2.  **Define Parameterized Projectors:** We define a parameterized unitary `U(theta)` and a static base projector $P_{base}$ of a specified size and rank.
3.  **Optimize Projectors via VQA:** For each local group of qubits, we run a simulated Variational Quantum Algorithm (VQA). A classical optimizer (COBYLA) seeks parameters `theta` to minimize the expectation value $\langle\psi_{initial}|P(theta)|\psi_{initial}\rangle$, which is calculated via simulated measurements.
4.  **Construct Parent Hamiltonian:** The Hamiltonian $H$ is built by summing the successfully optimized projectors $P^*$.
5.  **Verification:** We find the ground state of our discovered $H$ and verify that it has high fidelity with the original `|psi_initial>`, confirming our method worked.

## 1. Imports and Setup

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp, state_fidelity, random_unitary
import itertools 

try:
    from scipy.optimize import minimize
except ModuleNotFoundError:
    print("ERROR: scipy is not installed. Please install it: pip install scipy")
    def minimize(*args, **kwargs):
        raise RuntimeError("scipy.optimize.minimize not found. Please install scipy.")

## 2. Configuration Parameters

In [ ]:
# --- Main Experiment Configuration ---
NUM_QUBITS = 6
BOND_DIMENSION = 4        # Bond dimension of the initial MPS state
PROJECTOR_QUBIT_COUNT = 3  # Number of qubits each projector acts on

# Set projector rank based on projector size
# Rank 2**(N-1) is a projector onto the even parity subspace for the Z...Z operator
PROJECTOR_RANK = 2**(PROJECTOR_QUBIT_COUNT - 1)

# --- Optimizer and Ansatz Configuration ---
# Set unitary complexity based on projector size
PROJECTOR_U_REPS = PROJECTOR_QUBIT_COUNT - 1 
SCIPY_COBYLA_MAXITER = 2000
SCIPY_COBYLA_TOL = 1e-4
SEED = 42

np.random.seed(SEED)
print(f"--- Discovering Parent Hamiltonian for a Perfect MPS ---")
print(f"Config: Qubits={NUM_QUBITS}, Bond Dim={BOND_DIMENSION}")
print(f"Projectors: Size={PROJECTOR_QUBIT_COUNT}, Rank={PROJECTOR_RANK}, U_reps={PROJECTOR_U_REPS}\n")

## 3. Step 1: Generate a Perfect MPS State `|psi_initial>`
We generate the statevector by directly contracting a series of random tensors. This guarantees it is a perfect MPS with the specified `BOND_DIMENSION`.

In [ ]:
def generate_perfect_mps_statevector(num_qubits, bond_dim, physical_dim=2):
    """Generates a statevector from a random MPS tensor train."""
    mps_tensors = []
    # Left-most tensor (vector)
    rand_mat_first = np.random.randn(physical_dim, bond_dim) + 1j * np.random.randn(physical_dim, bond_dim)
    q_first, _ = np.linalg.qr(rand_mat_first.T)
    mps_tensors.append(q_first.T.reshape(physical_dim, 1, bond_dim))

    # Bulk tensors
    for _ in range(num_qubits - 2):
        rand_mat = np.random.randn(physical_dim * bond_dim, bond_dim) + 1j * np.random.randn(physical_dim * bond_dim, bond_dim)
        q_mid, _ = np.linalg.qr(rand_mat)
        mps_tensors.append(q_mid.reshape(physical_dim, bond_dim, bond_dim))

    # Right-most tensor (vector)
    rand_vec_last = np.random.randn(physical_dim * bond_dim) + 1j * np.random.randn(physical_dim * bond_dim)
    mps_tensors.append((rand_vec_last / np.linalg.norm(rand_vec_last)).reshape(physical_dim, bond_dim, 1))
    
    # Contract the MPS tensors to get the full statevector
    current_vector = mps_tensors[0][:, 0, :]
    for i in range(1, num_qubits):
        tensor = mps_tensors[i]
        current_vector = np.tensordot(current_vector, tensor, axes=([1], [1]))
        current_vector = current_vector.reshape(-1, tensor.shape[2])
    final_state_vector = current_vector.flatten()
    return Statevector(final_state_vector)

print("Step 1: Generating a perfect MPS state...")
psi_initial_vector = generate_perfect_mps_statevector(NUM_QUBITS, BOND_DIMENSION)
print(f"  |psi_initial> statevector created. Norm: {np.linalg.norm(psi_initial_vector.data):.4f}")

## 4. Step 2: Define and Optimize Projectors `P*` via VQA
We now use the optimization-based method to discover the projectors $P^*$ for which `<psi_initial|P*|psi_initial>` is minimal.

In [ ]:
print("\nStep 2: Defining and Optimizing Projectors based on |psi_initial>...")

# Define qubit groups with wrap-around
qubit_group_indices = [tuple((i + j) % NUM_QUBITS for j in range(PROJECTOR_QUBIT_COUNT)) for i in range(NUM_QUBITS)]
print(f"Qubit groups for projectors (with wrap-around): {qubit_group_indices}")

# Define unitary ansatz
u_ansatz_n_q = TwoLocal(PROJECTOR_QUBIT_COUNT, ['ry', 'rz'], 'cx', 'linear', reps=PROJECTOR_U_REPS)
num_u_params = u_ansatz_n_q.num_parameters

# Optimization Loop
optimized_projectors_P_star = []
for q_group in qubit_group_indices:
    q_group_str = ''.join(map(str, q_group))
    print(f"  Optimizing P_{q_group_str}(theta) acting on qubits {q_group}...")
    
    # Create base projector of specified rank
    base_projector_spo = SparsePauliOp("I" * NUM_QUBITS, coeffs=[0])
    for i in range(PROJECTOR_RANK):
        basis_state_str = format(i, f'0{PROJECTOR_QUBIT_COUNT}b')
        # Projector for |i><i| = product_k( |i_k><i_k| ) where |b><b|=(I+(-1)^b Z)/2
        current_proj_op = SparsePauliOp("I" * NUM_QUBITS, coeffs=[1.0])
        for local_idx, bit in enumerate(basis_state_str):
            q_idx = q_group[local_idx]
            z_op_str_list = ['I'] * NUM_QUBITS; z_op_str_list[q_idx] = 'Z'
            sign = 1.0 if bit == '0' else -1.0
            proj_term = SparsePauliOp("I"*NUM_QUBITS, coeffs=[0.5]) + SparsePauliOp("".join(z_op_str_list), coeffs=[0.5*sign])
            current_proj_op = current_proj_op.compose(proj_term)
        base_projector_spo += current_proj_op

    def cost_function_projector(params_theta):
        try:
            u_circuit_n_q = u_ansatz_n_q.assign_parameters(params_theta)
            u_full_qc = QuantumCircuit(NUM_QUBITS)
            u_full_qc.compose(u_circuit_n_q, qubits=list(q_group), inplace=True)
            u_full_op = Operator(u_full_qc)
            p_full_op_object = u_full_op @ base_projector_spo @ u_full_op.adjoint()
            return np.real(psi_initial_vector.expectation_value(p_full_op_object))
        except Exception:
            return 1e6 
    
    initial_u_params = np.random.rand(num_u_params) * 2 * np.pi
    try:
        opt_result = minimize(fun=cost_function_projector, x0=initial_u_params, method='COBYLA',
                              tol=SCIPY_COBYLA_TOL, options={'maxiter': SCIPY_COBYLA_MAXITER, 'disp': False})
        if opt_result.success or (not np.isnan(opt_result.fun) and opt_result.fun < 1.0):
            u_circuit_n_q_opt = u_ansatz_n_q.assign_parameters(opt_result.x)
            u_full_qc_opt = QuantumCircuit(NUM_QUBITS)
            u_full_qc_opt.compose(u_circuit_n_q_opt, qubits=list(q_group), inplace=True)
            u_full_op_opt = Operator(u_full_qc_opt)
            p_star_op_object = u_full_op_opt @ base_projector_spo @ u_full_op_opt.adjoint()
            optimized_projectors_P_star.append(p_star_op_object)
            print(f"    Successfully optimized P_{q_group_str}*. Min value: {opt_result.fun:.6e}")
        else:
            print(f"    Warning: Optimization for P_{q_group_str}* failed. Message: {opt_result.message}")
    except Exception as e_general_opt:
        print(f"    ERROR: General error during optimization for P_{q_group_str}*: {e_general_opt}")

## 5. Step 3: Find Ground State of Parent Hamiltonian and Test Claim
We construct the Hamiltonian $H = \sum P^*$ and find its exact ground state `|psi_GS>`. The claim is tested by computing the fidelity between `|psi_initial>` and `|psi_GS>`.

In [ ]:
print("\nStep 3: Finding Ground State of H = sum(P*) and Verifying Claim...")

if not optimized_projectors_P_star:
    print("Warning: No optimized projectors were created. Cannot test the claim.")
else:
    print("Constructing Hamiltonian H = sum(P*)...")
    hamiltonian_h_operator = Operator(np.zeros((2**NUM_QUBITS, 2**NUM_QUBITS)))
    for p_star in optimized_projectors_P_star:
        hamiltonian_h_operator += p_star
    
    print("Diagonalizing Hamiltonian matrix...")
    h_matrix = hamiltonian_h_operator.data
    eigenvalues, eigenvectors = np.linalg.eigh(h_matrix)
    
    ground_state_energy = eigenvalues[0]
    psi_gs_vector = Statevector(eigenvectors[:, 0])

    print(f"  Hamiltonian constructed. Ground State Energy (H_min): {ground_state_energy:.6e}")
    
    # Check for degeneracy
    if len(eigenvalues) > 1:
        energy_gap = eigenvalues[1] - eigenvalues[0]
        print(f"  Energy gap to first excited state: {energy_gap:.6f}")
        if energy_gap > 1e-9:
            print("  Ground state appears to be UNIQUE (non-degenerate).")
        else:
            print("  Ground state appears to be DEGENERATE.")
    else:
        print("  Hamiltonian has only one eigenvalue. Ground state is UNIQUE.")
        
    # --- VERIFICATION OF THE CLAIM ---
    print("\n--- Claim Test Result ---")
    try:
        # Normalize both vectors for a fair comparison
        norm_initial = np.linalg.norm(psi_initial_vector.data)
        norm_gs = np.linalg.norm(psi_gs_vector.data)
        normalized_initial_data = psi_initial_vector.data / norm_initial
        normalized_gs_data = psi_gs_vector.data / norm_gs
        
        fidelity = state_fidelity(normalized_initial_data, normalized_gs_data)
        
        print(f"Fidelity( |psi_initial>, |psi_GS> ): {fidelity:.8f}")
        
        if fidelity > 0.999:
            print("\nConclusion: CLAIM SUPPORTED. The perfect MPS state is the unique ground state of the Hamiltonian discovered via optimization.")
        else:
            print("\nConclusion: CLAIM NOT SUPPORTED. The ground state of the discovered H is different from the initial MPS.")
            print("This is likely due to the optimizer finding sub-optimal local minima for some projectors.")
            
    except Exception as e:
        print(f"An error occurred during fidelity calculation: {e}")